In [6]:
import os
import json
import pandas as pd

In [7]:
DATA_PATH = os.path.join("..","data", "raw", "cvm_jsons")

In [8]:
def extract_info(filepath):
    with open(filepath, "r", encoding="utf-8") as f:
        data = json.load(f)

    info = data.get("informacoesGerais", {})
    grupos = data.get("grupos", [])

    rows = []
    for grupo in grupos:
        for serie in grupo.get("series", []):
            lote_final = serie.get("loteFinal", {})
            lote_inicial = serie.get("loteInicial", {})

            # Campos adicionais dentro de "camposCadastrados"
            campos = {c["campoNome"]: c.get("campoValor") for c in lote_final.get("camposCadastrados", [])}

            row = {
                # Infos gerais
                "numeroProcesso": info.get("numeroProcesso"),
                "numeroRegistro": serie.get("numeroRegistro", info.get("numeroRegistro")),
                "nomeValorMobiliario": info.get("nomeValorMobiliario"),
                "valorTotalInicial": info.get("valorTotalInicial"),
                "valorTotalFinal": info.get("valorTotalFinal"),
                "dataOferta": info.get("data"),
                "nomeTipoRequerimento": info.get("nomeTipoRequerimento"),
                "status": info.get("status"),

                # Infos da série
                "serie_nome": lote_final.get("valorMobiliario", lote_inicial.get("valorMobiliario")),
                "quantidadeAtivos": lote_final.get("loteBase", {}).get("quantidadeAtivos"),
                "valorNominal": lote_final.get("loteBase", {}).get("valorNominal"),
                "valorTotalLote": lote_final.get("valorTotalLote"),
                "taxaRemuneracao": lote_final.get("taxaRemuneracao"),

                # Campos cadastrados principais
                "dataEmissao": campos.get("Data de emissão"),
                "dataVencimento": campos.get("Data de vencimento"),
                "especie": campos.get("Espécie"),
                "avaliacaoRisco": campos.get("Avaliação de risco"),
                "remuneracaoFinal": campos.get("Informações sobre remuneração final (pós bookbuilding)"),
                "amortizacao": campos.get("Informações sobre amortização"),
                "incentivado": campos.get("Título incentivado - Lei 12.431/11"),
                "infraLei14801": campos.get("Debêntures de infraestrutura - Lei 14.801/24"),
                "resgateAntecipado": campos.get("Possibilidade de resgate antecipado")
            }

            rows.append(row)

    # Se não houver grupos/séries, ainda retorna linha com infos gerais
    if not rows:
        rows.append({
            "numeroProcesso": info.get("numeroProcesso"),
            "numeroRegistro": info.get("numeroRegistro"),
            "nomeValorMobiliario": info.get("nomeValorMobiliario"),
            "valorTotalInicial": info.get("valorTotalInicial"),
            "valorTotalFinal": info.get("valorTotalFinal"),
            "dataOferta": info.get("data"),
            "nomeTipoRequerimento": info.get("nomeTipoRequerimento"),
            "status": info.get("status"),
            "serie_nome": None,
            "quantidadeAtivos": None,
            "valorNominal": None,
            "valorTotalLote": None,
            "taxaRemuneracao": None,
            "dataEmissao": None,
            "dataVencimento": None,
            "especie": None,
            "avaliacaoRisco": None,
            "remuneracaoFinal": None,
            "amortizacao": None,
            "incentivado": None,
            "infraLei14801": None,
            "resgateAntecipado": None
        })

    return rows

In [9]:
files = [f for f in os.listdir(DATA_PATH) if f.endswith(".json")]

In [10]:
all_rows = []
for file in files:
    filepath = os.path.join(DATA_PATH, file)
    all_rows.extend(extract_info(filepath))

df = pd.DataFrame(all_rows)
print(df.head(10))  # mostra as primeiras linhas
df.to_csv("cvm_ofertas_expandidas.csv", index=False, encoding="utf-8-sig")
print("✅ DataFrame salvo em cvm_ofertas_expandidas.csv")

  numeroProcesso                numeroRegistro nomeValorMobiliario  \
0  SRE/1820/2025  CVM/SRE/AUT/DEB/PRI/2025/655          Debêntures   
1  SRE/2275/2025  CVM/SRE/AUT/DEB/PRI/2025/648          Debêntures   
2  SRE/2275/2025  CVM/SRE/AUT/DEB/PRI/2025/649          Debêntures   
3  SRE/2275/2025  CVM/SRE/AUT/DEB/PRI/2025/650          Debêntures   
4  SRE/2553/2025  CVM/SRE/AUT/DEB/PRI/2025/661          Debêntures   
5  SRE/2553/2025  CVM/SRE/AUT/DEB/PRI/2025/662          Debêntures   
6  SRE/2551/2025  CVM/SRE/AUT/DEB/PRI/2025/657          Debêntures   
7  SRE/2551/2025  CVM/SRE/AUT/DEB/PRI/2025/658          Debêntures   
8  SRE/2552/2025  CVM/SRE/AUT/DEB/PRI/2025/659          Debêntures   
9  SRE/2552/2025  CVM/SRE/AUT/DEB/PRI/2025/660          Debêntures   

    valorTotalInicial     valorTotalFinal  dataOferta  \
0    300.000.000,0000    300.000.000,0000  12/09/2025   
1  1.600.000.000,0000  1.600.000.000,0000  11/09/2025   
2  1.600.000.000,0000  1.600.000.000,0000  11/09/2025   
3